#Boltzmann Machine

##Downloading the dataset

###ML-100K

###ML-1M

##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable

## Importing the dataset


In [2]:
users= pd.read_csv('users.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')
movies= pd.read_csv('movies.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')
ratings= pd.read_csv('ratings.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')

## Preparing the training set and the test set


In [3]:
training_data= pd.read_csv('u1.base', delimiter='\t')
training_data= np.array(training_data, dtype='int')
test_data= pd.read_csv('u1.test', delimiter='\t')
test_data= np.array(training_data, dtype='int')

## Getting the number of users and movies


In [4]:
nb_users= max(max(training_data[:,0]),max(test_data[:,0]))
nb_movies=  max(max(training_data[:,1]),max(test_data[:,1]))

## Converting the data into an array with users in lines and movies in columns


In [5]:
def convert(data):
  new_data=[]
  for userID in range(1, nb_users+1):
    IDmovies= data[:,1][data[:, 0]== userID]
    IDratings= data[:, 2][data[:, 0]== userID]
    ratings= np.zeros(nb_movies)
    ratings[IDmovies -1]= IDratings
    new_data.append(list(ratings))
  return new_data
training_data= convert(training_data)
test_data= convert(test_data)

## Converting the data into Torch tensors


In [6]:
training_data= torch.FloatTensor(training_data) 
test_data= torch.FloatTensor(test_data)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [7]:
training_data[training_data==0]=-1
training_data[training_data==1]=0
training_data[training_data==2]=0
training_data[training_data>=3]=1
test_data[test_data==0]=-1
test_data[test_data==1]=0
test_data[test_data==2]=0
test_data[test_data>=3]=1

## Creating the architecture of the Neural Network


In [8]:
class RBM():
  def __init__(self, nv, nh):
      self.W= torch.randn(nh, nv)
      self.a= torch.randn(1, nh)
      self.b= torch.randn(1, nv)
  def sample_h(self,x):
      wx= torch.mm(x, self.W.t())
      activation= wx+ self.a.expand_as(wx)
      p_h_given_v= torch.sigmoid(activation)
      return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self,y):
      wy= torch.mm(y, self.W)
      activation= wy+ self.b.expand_as(wy)
      p_v_given_h= torch.sigmoid(activation)
      return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
      self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
      self.a += torch.sum((ph0-phk), 0)
      self.b += torch.sum((v0-vk), 0)
nv= len(training_data[0])
nh= 100
batch_size= 100
rbm= RBM( nv, nh)

## Training the RBM


In [9]:
nb_epoch=10

for epoch in range(1, nb_epoch+1):
  train_loss=0
  s=0. 
  for id_user in range(0, nb_users-batch_size, batch_size):
    vk= training_data[id_user:id_user+batch_size]
    v0= training_data[id_user:id_user+batch_size]
    ph0, _= rbm.sample_h(v0)
    for k in range(10):
      _,hk= rbm.sample_h(vk)
      _,vk= rbm.sample_v(hk)
      vk[v0<0]= v0[v0<0]
    phk, _= rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss= torch.mean( torch.abs(v0[ v0>=0] - vk[ v0>=0]))
    s+=1. 
  print('epoch: '+str(epoch)+' training_loss '+str(train_loss/s))

epoch: 1 training_loss tensor(0.0288)
epoch: 2 training_loss tensor(0.0315)
epoch: 3 training_loss tensor(0.0299)
epoch: 4 training_loss tensor(0.0285)
epoch: 5 training_loss tensor(0.0286)
epoch: 6 training_loss tensor(0.0278)
epoch: 7 training_loss tensor(0.0269)
epoch: 8 training_loss tensor(0.0277)
epoch: 9 training_loss tensor(0.0283)
epoch: 10 training_loss tensor(0.0279)


## Testing the RBM


In [10]:
train_loss=0
s=0. 
for id_user in range(nb_users):
  v= training_data[id_user:id_user+1]
  vt= test_data[id_user:id_user+1]
  ph0, _= rbm.sample_h(v0)
  if len(vt[ vt>=0])>0:
    _,h= rbm.sample_h(v)
    _,v= rbm.sample_v(h)
    test_loss= torch.mean( torch.abs(vt[ vt>=0] - v[ vt>=0]))
    s+=1. 
print('Test_loss '+str(test_loss/s))

Test_loss tensor(0.0003)
